In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as sk
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(15, input_dim=15, kernel_initializer='normal', activation='relu'))
	model.add(Dense(15, input_dim=15, kernel_initializer='normal', activation='relu'))
	model.add(Dense(2, kernel_initializer='normal'))
		# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dati/dataset_taggato.csv', sep = ';')

In [ ]:
x = df.iloc[: , 1:16]
scaler = StandardScaler()
cols = x.columns
x[cols] = x[cols].replace(',','.', regex = True)
x[cols] = x[cols].apply(pd.to_numeric,downcast='float', errors='raise')
x = x.astype('float64')

x = pd.DataFrame(scaler.fit_transform(x.values), columns=x.columns, index=x.index)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
y = df.iloc[: , 28:30]
cols = y.columns
y[cols] = y[cols].replace(',','.', regex = True)

y[cols] = y[cols].apply(pd.to_numeric,downcast='float', errors='raise')

y = pd.DataFrame(scaler2.fit_transform(y.values), columns=y.columns, index=y.index)

display(y.head(5))
print(y.dtypes)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
model=baseline_model()
estimator = KerasRegressor(model, epochs=10, batch_size=5, verbose=1)
kfold = KFold(n_splits=10) 
results = cross_val_score(estimator, x_train, y_train, cv=kfold, verbose = 5, n_jobs=-1)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))


In [ ]:
estimator.fit(x_train, y_train)
preds = estimator.predict(x_test)


In [ ]:
r_sq = estimator.score(x_train, y_train)
print('R^2:', r_sq)

adj_r2 = 1 - (1-r_sq)*(len(y_train)-1)/(len(y_train)-x_train.shape[1]-1)
print('adjusted R^2:', adj_r2)

In [ ]:
y_pred = estimator.predict(x_test)
mse = mean_squared_error(scaler2.inverse_transform(y_test), scaler2.inverse_transform(y_pred), multioutput = "raw_values")
r2_scores = sk.r2_score(y_test, y_pred, multioutput = "raw_values")

In [ ]:
print ("r2:", r2_scores)
print ("mse: ", mse)

In [ ]:
estimator.model.save("/content/sample_data/my_model")

In [ ]:
!zip -r /content/sample_data/my_model.zip /content/sample_data/my_model

In [ ]:
from google.colab import files
files.download('/content/sample_data/my_model.zip')

In [ ]:
!kill -9 -1